# BGP Anomaly Detection with Normal Baseline

## The Correct Approach for Anomaly Detection

**Problem with previous approach:**
- Unsupervised methods find outliers *within* the provided data
- If you only provide attack data, they find "unusual attacks" not "attacks vs normal"

**Correct approach:**
1. Train models on **NORMAL traffic** (learn what normal looks like)
2. Score **NEW/UNKNOWN data** against the normal baseline
3. High scores = anomalies (deviates from normal)

---

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.covariance import EllipticEnvelope
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')
print("Imports successful!")

In [ ]:
# ============================================
# CONFIGURATION
# ============================================

# NORMAL traffic file (for training the baseline)
NORMAL_FILE = "/path/to/normal_traffic_features.csv"  # <-- YOUR NORMAL DATA

# File to test/score (can be normal, attack, or unknown)
TEST_FILE = "/path/to/test_features.csv"  # <-- DATA TO CLASSIFY

# Output directory
OUTPUT_DIR = None  # Uses test file directory if None

# Label column (if exists in test file)
LABEL_COL = 'label'

# Threshold for anomaly (percentile of normal data scores)
# 95 means: flag as anomaly if score is worse than 95% of normal data
ANOMALY_THRESHOLD_PERCENTILE = 95

# Random seed
RANDOM_STATE = 42

# Save plots
SAVE_PLOTS = True

print("Configuration set")
print(f"  Normal file: {NORMAL_FILE}")
print(f"  Test file: {TEST_FILE}")

## 1. Load Data

In [ ]:
# Load normal data (for training)
print("Loading NORMAL data (for baseline training)...")
df_normal = pd.read_csv(NORMAL_FILE)
print(f"  Loaded {len(df_normal)} normal samples")

# Load test data (to classify)
print("\nLoading TEST data (to classify)...")
df_test = pd.read_csv(TEST_FILE)
print(f"  Loaded {len(df_test)} test samples")

# Show test data label distribution if available
if LABEL_COL and LABEL_COL in df_test.columns:
    print(f"\nTest data labels (ground truth):")
    for label, count in df_test[LABEL_COL].value_counts().items():
        print(f"  {label}: {count} ({count/len(df_test)*100:.1f}%)")

In [ ]:
# Identify feature columns - exclude metadata and previous analysis columns
META_COLS = {
    # Metadata columns
    'incident', 'window_start', 'window_end', 'timestamp', 'time',
    'label', 'label_rule', 'label_refined', 'source', 'collector',
    # Previous analysis columns (from BGP_Label_Validation_Discovery.ipynb)
    'iso_forest_score', 'lof_score', 'statistical_score', 'elliptic_score',
    'cluster', 'anomaly_votes', 'consensus_score', 'discovered_label',
    # Any other score columns
    'hdbscan_score', 'predicted_label'
}

def get_feature_cols(df):
    """Get only the actual BGP feature columns, excluding metadata and analysis columns."""
    candidates = [c for c in df.columns if c.lower() not in {m.lower() for m in META_COLS}]
    return df[candidates].select_dtypes(include=[np.number]).columns.tolist()

# Get feature columns from BOTH datasets and use intersection
normal_features = set(get_feature_cols(df_normal))
test_features = set(get_feature_cols(df_test))

# Use only features present in BOTH datasets
feature_cols = sorted(list(normal_features & test_features))

print(f"Normal dataset has {len(normal_features)} features")
print(f"Test dataset has {len(test_features)} features")
print(f"Using {len(feature_cols)} common features")
print(f"\nFeatures: {feature_cols}")

# Prepare normal data
X_normal = df_normal[feature_cols].values
valid_normal = ~np.isnan(X_normal).any(axis=1)
X_normal = X_normal[valid_normal]
print(f"\nNormal samples after removing NaN: {len(X_normal)}")

# Prepare test data
X_test = df_test[feature_cols].values
valid_test = ~np.isnan(X_test).any(axis=1)
X_test_clean = X_test[valid_test]
print(f"Test samples after removing NaN: {len(X_test_clean)}")

# Fit scaler on NORMAL data only
scaler = RobustScaler()
X_normal_scaled = scaler.fit_transform(X_normal)
X_test_scaled = scaler.transform(X_test_clean)  # Transform test with normal's scaler

print("\nScaler fitted on NORMAL data and applied to test data")

## 2. Train Models on NORMAL Data

This is the key difference: we train **only on normal data** to learn what normal looks like.

In [ ]:
print("Training anomaly detection models on NORMAL data...")
print("="*60)

models = {}
normal_scores = {}

# 1. Isolation Forest
print("\n[1/4] Training Isolation Forest...")
models['IsolationForest'] = IsolationForest(
    n_estimators=200,
    contamination=0.01,  # Expect very few anomalies in normal data
    random_state=RANDOM_STATE,
    n_jobs=-1
)
models['IsolationForest'].fit(X_normal_scaled)
normal_scores['IsolationForest'] = models['IsolationForest'].decision_function(X_normal_scaled)
print(f"  Normal score range: [{normal_scores['IsolationForest'].min():.3f}, {normal_scores['IsolationForest'].max():.3f}]")

# 2. Local Outlier Factor (novelty detection mode)
print("\n[2/4] Training Local Outlier Factor...")
n_neighbors = max(10, min(50, int(np.sqrt(len(X_normal_scaled)))))
models['LOF'] = LocalOutlierFactor(
    n_neighbors=n_neighbors,
    contamination=0.01,
    novelty=True  # Important: enables scoring new data
)
models['LOF'].fit(X_normal_scaled)
normal_scores['LOF'] = models['LOF'].decision_function(X_normal_scaled)
print(f"  Normal score range: [{normal_scores['LOF'].min():.3f}, {normal_scores['LOF'].max():.3f}]")

# 3. Elliptic Envelope
print("\n[3/4] Training Elliptic Envelope...")
try:
    models['EllipticEnvelope'] = EllipticEnvelope(
        contamination=0.01,
        random_state=RANDOM_STATE
    )
    models['EllipticEnvelope'].fit(X_normal_scaled)
    normal_scores['EllipticEnvelope'] = models['EllipticEnvelope'].decision_function(X_normal_scaled)
    print(f"  Normal score range: [{normal_scores['EllipticEnvelope'].min():.3f}, {normal_scores['EllipticEnvelope'].max():.3f}]")
except Exception as e:
    print(f"  Skipped due to error: {e}")
    models['EllipticEnvelope'] = None

# 4. Statistical baseline (mean and std of normal data)
print("\n[4/4] Computing Statistical baseline...")
models['Statistical'] = {
    'mean': np.mean(X_normal_scaled, axis=0),
    'std': np.std(X_normal_scaled, axis=0)
}
# Mahalanobis-like distance from normal mean
normal_scores['Statistical'] = -np.mean(
    np.abs((X_normal_scaled - models['Statistical']['mean']) / (models['Statistical']['std'] + 1e-10)), 
    axis=1
)
print(f"  Normal score range: [{normal_scores['Statistical'].min():.3f}, {normal_scores['Statistical'].max():.3f}]")

print("\n" + "="*60)
print("All models trained on NORMAL data!")

In [ ]:
# Compute thresholds based on normal data distribution
print(f"Computing anomaly thresholds ({ANOMALY_THRESHOLD_PERCENTILE}th percentile of normal scores)...")

thresholds = {}
for method, scores in normal_scores.items():
    # For decision_function: higher = more normal, lower = more anomalous
    # So we use a LOW percentile as threshold
    thresholds[method] = np.percentile(scores, 100 - ANOMALY_THRESHOLD_PERCENTILE)
    print(f"  {method}: threshold = {thresholds[method]:.4f}")

print("\nSamples scoring BELOW these thresholds will be flagged as anomalies")

## 3. Score Test Data Against Normal Baseline

In [ ]:
print("Scoring TEST data against NORMAL baseline...")
print("="*60)

test_scores = {}
test_anomalies = {}

# 1. Isolation Forest
print("\n[1/4] Scoring with Isolation Forest...")
test_scores['IsolationForest'] = models['IsolationForest'].decision_function(X_test_scaled)
test_anomalies['IsolationForest'] = test_scores['IsolationForest'] < thresholds['IsolationForest']
print(f"  Anomalies: {test_anomalies['IsolationForest'].sum()} ({test_anomalies['IsolationForest'].mean()*100:.1f}%)")

# 2. LOF
print("\n[2/4] Scoring with LOF...")
test_scores['LOF'] = models['LOF'].decision_function(X_test_scaled)
test_anomalies['LOF'] = test_scores['LOF'] < thresholds['LOF']
print(f"  Anomalies: {test_anomalies['LOF'].sum()} ({test_anomalies['LOF'].mean()*100:.1f}%)")

# 3. Elliptic Envelope
if models['EllipticEnvelope'] is not None:
    print("\n[3/4] Scoring with Elliptic Envelope...")
    test_scores['EllipticEnvelope'] = models['EllipticEnvelope'].decision_function(X_test_scaled)
    test_anomalies['EllipticEnvelope'] = test_scores['EllipticEnvelope'] < thresholds['EllipticEnvelope']
    print(f"  Anomalies: {test_anomalies['EllipticEnvelope'].sum()} ({test_anomalies['EllipticEnvelope'].mean()*100:.1f}%)")

# 4. Statistical
print("\n[4/4] Scoring with Statistical method...")
test_scores['Statistical'] = -np.mean(
    np.abs((X_test_scaled - models['Statistical']['mean']) / (models['Statistical']['std'] + 1e-10)), 
    axis=1
)
test_anomalies['Statistical'] = test_scores['Statistical'] < thresholds['Statistical']
print(f"  Anomalies: {test_anomalies['Statistical'].sum()} ({test_anomalies['Statistical'].mean()*100:.1f}%)")

print("\n" + "="*60)

In [ ]:
# Compute consensus
print("Computing consensus across methods...")

n_methods = len(test_anomalies)
anomaly_votes = np.zeros(len(X_test_scaled))

for method, anomalies in test_anomalies.items():
    anomaly_votes += anomalies.astype(int)

consensus_score = anomaly_votes / n_methods

# Assign labels
predicted_labels = np.where(
    consensus_score >= 0.75, 'anomaly',
    np.where(
        consensus_score >= 0.5, 'likely_anomaly',
        np.where(
            consensus_score >= 0.25, 'uncertain',
            'normal'
        )
    )
)

print("\nPredicted Label Distribution:")
print("="*50)
for label in ['normal', 'uncertain', 'likely_anomaly', 'anomaly']:
    count = (predicted_labels == label).sum()
    pct = count / len(predicted_labels) * 100
    emoji = "🟢" if label == 'normal' else "🔴" if 'anomaly' in label else "🟡"
    print(f"  {emoji} {label}: {count} ({pct:.1f}%)")

## 4. Evaluate Against Ground Truth (if available)

In [ ]:
if LABEL_COL and LABEL_COL in df_test.columns:
    print("Evaluating against ground truth labels...")
    print("="*60)
    
    # Get ground truth for valid samples
    ground_truth = df_test.loc[valid_test, LABEL_COL].values
    
    # Binary ground truth: normal vs not normal
    is_attack_gt = ~np.isin(ground_truth, ['normal', 'Normal', 'NORMAL'])
    is_attack_pred = np.isin(predicted_labels, ['anomaly', 'likely_anomaly'])
    
    # Calculate metrics
    TP = np.sum(is_attack_gt & is_attack_pred)
    TN = np.sum(~is_attack_gt & ~is_attack_pred)
    FP = np.sum(~is_attack_gt & is_attack_pred)
    FN = np.sum(is_attack_gt & ~is_attack_pred)
    
    accuracy = (TP + TN) / len(ground_truth)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"\n📊 DETECTION PERFORMANCE:")
    print(f"  Accuracy:  {accuracy*100:.1f}%")
    print(f"  Precision: {precision*100:.1f}%")
    print(f"  Recall:    {recall*100:.1f}%")
    print(f"  F1-Score:  {f1*100:.1f}%")
    
    print(f"\n📋 CONFUSION MATRIX:")
    print(f"  True Positives (attacks detected):  {TP}")
    print(f"  True Negatives (normal correct):    {TN}")
    print(f"  False Positives (false alarms):     {FP}")
    print(f"  False Negatives (missed attacks):   {FN}")
    
    # Per-class detection rates
    print(f"\n📈 PER-CLASS DETECTION RATES:")
    for label in df_test[LABEL_COL].unique():
        mask = ground_truth == label
        if mask.sum() > 0:
            detected = is_attack_pred[mask].sum()
            rate = detected / mask.sum() * 100
            emoji = "✅" if rate > 80 else "⚠️" if rate > 50 else "❌"
            print(f"  {emoji} {label}: {detected}/{mask.sum()} detected ({rate:.1f}%)")
else:
    print("No ground truth labels available for evaluation.")

In [ ]:
# Visualize score distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, (method, scores) in enumerate(test_scores.items()):
    if i >= 4:
        break
    
    ax = axes[i]
    
    # Plot normal scores distribution
    ax.hist(normal_scores[method], bins=50, alpha=0.5, label='Normal (training)', color='green', density=True)
    
    # Plot test scores distribution
    ax.hist(scores, bins=50, alpha=0.5, label='Test data', color='blue', density=True)
    
    # Mark threshold
    ax.axvline(x=thresholds[method], color='red', linestyle='--', linewidth=2, label=f'Threshold')
    
    ax.set_xlabel('Anomaly Score')
    ax.set_ylabel('Density')
    ax.set_title(f'{method}\n(Anomalies: {test_anomalies[method].mean()*100:.1f}%)')
    ax.legend()

plt.tight_layout()
plt.savefig('score_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

print("Plot saved to: score_distributions.png")

In [ ]:
# Save results
results_df = df_test.loc[valid_test].copy()

# Add scores
for method, scores in test_scores.items():
    results_df[f'{method}_score'] = scores
    results_df[f'{method}_anomaly'] = test_anomalies[method]

results_df['consensus_score'] = consensus_score
results_df['predicted_label'] = predicted_labels

# Save
output_path = Path(TEST_FILE).parent / f"{Path(TEST_FILE).stem}_anomaly_detection.csv"
results_df.to_csv(output_path, index=False)
print(f"Results saved to: {output_path}")

## Summary

This notebook implements the **correct** approach for anomaly detection:

1. **Train on NORMAL data** - Learn what normal traffic looks like
2. **Compute thresholds** - Based on normal data distribution
3. **Score test data** - Measure how different from normal
4. **Flag anomalies** - Samples that deviate significantly from normal

This approach will correctly identify attacks as anomalies because they differ from the normal baseline.